# Clustering and Scoring Job Relocation Opportunities - ETL Scripts

Austin Rainwater

---

For this notebook, I have created some libraries to assist in building an Extract/Transform/Load Data Pipeline. This pipeline uses asyncio, so that multiple requests to different endpoints can be made at one time. The pipeline allows the full Extract/Transform/Load process to be run concurrently, instead of synchronously running each step and waiting for their results.

The pipeline consists of an abstract `PipelineStep` class. Each step must define a `process_batch(self, batch)` coroutine. 

Before running my imports, I'm going to run `test.py`, which tests to make sure the libraries are working as intended.

In [1]:
!python test.py

..............
----------------------------------------------------------------------
Ran 14 tests in 1.224s

OK


In [2]:
!pip install --quiet --upgrade sqlalchemy==1.3.22 pymysql==0.9.3 PyYAML aiohttp aiomysql==0.0.21

from etl import PipelineStep, DataPipeline
from datetime import datetime, timedelta
from warnings import warn
from abc import ABC
from itertools import product
from concurrent.futures import ProcessPoolExecutor
from math import ceil
import warnings
import xml.etree.ElementTree as xml
import re
import yaml

import numpy as np
import pandas as pd
from pandas import json_normalize

import asyncio
import aiohttp

import sqlalchemy as sa
from aiomysql.sa import create_engine
from aiomysql import Warning as MariaDBWarning
from sqlalchemy import (
    Table,
    Column,
    MetaData,
    String,
    Numeric,
    func,
    select
)
count = func.count
from sqlalchemy.schema import CreateTable, DropTable
from sqlalchemy.dialects.mysql import YEAR as Year, INTEGER as Integer, insert

with open('secrets.yaml', 'r') as secrets_file:
    secrets = yaml.safe_load(secrets_file)

## PipelineStep Subclasses

These are some PipelineStep subclasses I plan to use multiple times because I will be accessing specific APIs.

In [3]:
class RequestTracker:
    """Helper class to make sure we don't consistently go over API limits.
    
    Tracks the timestamp of requests via `track()`. 
    
    Attributes
    ----------
        request_times : list
            The datetimes that a request was tracked (i.e. RequestTracker.track()) was called
        
        default_delta : datetime.timedelta
            The default time limit to use for time-based methods. Defaults to 1 hour.
            
        default_limit : int
            The default number of requests to allow in the specified time period.
    """
    
    
    def __init__(self, default_delta=timedelta(hours=1), default_limit=5000):
        self.request_times = []
        self.default_delta = default_delta
        self.default_limit = default_limit
        
    @property
    def total_count(self):
        """The total number of requests made, regardless of time."""
        return len(self.request_times)
    
    def track(self):
        """Log that a request was made and should count against rate limits"""
        self.request_times.append(datetime.now())
    
    def requests_in_past(self, delta=None, **kwargs):
        """Counts the number of requests tracked in the time provided.
        
        Arguments
        ---------
            delta : timedelta
                How long requests should be considered against the rate limit. Can 
                alternatively be set using keyword arguments accepted by datetime.timedelta,
                i.e. microseconds, seconds, minutes, hours, etc.
                
        Examples
        --------
        >>> tracker = RequestTracker()
        >>> tracker.requests_in_past()
        0

        >>> tracker.track()
        >>> tracker.track()
        >>> tracker.track()
        >>> delta = datetime.timedelta(hours=3)
        >>> tracker.requests_in_past(delta)  # Assuming this didn't take 3 hours to run
        3

        >>> time.sleep(15)
        >>> tracker.requests_in_past(seconds=14)
        0
            
        """
        if not delta:
            delta = timedelta(**kwargs) if kwargs != {} else self.default_delta
        target_time = datetime.now() - delta
        return len([t for t in self.request_times if t > target_time])
    
    def is_ready(self, limit=None, delta=None, **kwargs):
        """Returns if requests are allowed, given the provided rate limits.
        
        Arguments
        ---------
            limit : int
                The number of requests to count before a subsequent request would be 
                considered "over the limit" (i.e. the request returns True)
            
            delta : timedelta
                How long requests should be considered against the rate limit. Can 
                alternatively be set using keyword arguments accepted by datetime.timedelta,
                i.e. microseconds, seconds, minutes, hours, etc.
                
                
        Examples
        --------
        >>> tracker = RequestTracker()
        >>> tracker.is_ready(limit=50, hours=1)
        True
        
        >>> _ = [tracker.track()] * 100
        >>> tracker.is_ready(limit=100, hours=1)
        False
        
        >>> time.sleep(15)
        >>> tracker.is_ready(limit=100, seconds=14)
        True
        
        """
        if not limit:
            limit = self.default_limit
        return self.requests_in_past(delta, **kwargs) < limit
    
    async def throttle(self, limit=None, delta=None, **kwargs):
        """Coroutine to block execution until is_ready returns True"""
        while not self.is_ready(limit, delta, **kwargs):
            await asyncio.sleep(0)  # Yield to event loop, but otherwise try again immediately
    

In [4]:
class WikipediaPipelineStep(PipelineStep, ABC):
    """
    PipelineStep with some helper methods to request data from Wikipedia
    """
    request_counter = RequestTracker()
    wikipedia_url = 'https://en.wikipedia.org/w/api.php'
    wikipedia_header = {"User-Agent": 
          'datascience jupyter notebook/0.2 '
          '(https://github.com/pacorain/datascience-certification-final-project; '
          'Austin Rainwater, paco@heckin.io)'}
    max_batch_size = 1
    async_batches = True
    
    def start(self):
        self.session = aiohttp.ClientSession()
        super().start()
        
    def stop(self):
        super().stop()
        loop = asyncio.get_running_loop()
        loop.create_task(self.session.close())
    
    async def make_request(self, params, fail_delay=0.46875):
        params['maxlag'] = 5  # see https://www.mediawiki.org/wiki/Manual:Maxlag_parameter
        
        await self.request_counter.throttle(limit=100, seconds=5) 
        # ^^ This is not a limit set by Wikipedia, but they do request developers be considerate.
        async with self.session.get(self.wikipedia_url, params=params, headers=self.wikipedia_header) as request:
            self.request_counter.track()
            response = await request.json()
            if 'error' in response and response['error']['code'] == 'maxlag':
                warnings.warn(f"Wikipedia is lagging. Trying again in {ceil(fail_delay)} second(s)")
                await asyncio.sleep(ceil(fail_delay))
                return await self.make_request(params, fail_delay=min(fail_delay * 2, 28800))
            assert 200 <= request.status <= 299
            if params['action'] == 'query' and 'continue' in response:
                warn("A Continue was issued but not handled.")
                # I'm not sure how to integrate this, or if I will even need to.
            return response

In [5]:
class FoursquarePipelineStep(PipelineStep, ABC):
    request_counter = RequestTracker(timedelta(hours=1), 5000)
    categories = None
    foursquare_header = {
        "User-Agent": 
          'datascience jupyter notebook/0.2 '
          '(https://github.com/pacorain/datascience-certification-final-project; '
          'Austin Rainwater, paco@heckin.io)'
    }
    api_base_url = 'https://api.foursquare.com/v2/venues/{}'
    v = '20201108'
    async_batches = True
    rate_block = asyncio.Lock()
    
        
    def start(self):
        self.session = aiohttp.ClientSession(headers=self.foursquare_header)
        super().start()
        
    def stop(self):
        super().stop()
        loop = asyncio.get_running_loop()
        loop.create_task(self.session.close())
        
    async def make_request(self, endpoint, *args, fail_delay=0.46875, ignore_lock=False, **params):
        
        if args and isinstance(args[0], dict):
            params = args[0]
        params['v'] = self.v
        params['client_id'] = secrets['4SQ_CLIENT_ID']
        params['client_secret'] = secrets['4SQ_CLIENT_SECRET']
        url = self.api_base_url.format(endpoint)
        if not self.request_counter.is_ready():
            warn("Foursquare API has hit hourly limit.")
            await self.request_counter.throttle()
        if not self.request_counter.is_ready(limit=99500, hours=24):
            warn("Foursquare API has hit daily limit.")
            await self.request_counter.throttle(limit=99500, hours=24)
            
        # Block execution while a rate-limited request retries
        if not ignore_lock and self.rate_block.locked():
            await self.rate_block.acquire()
            # Once one request succeeds, we don't need to block other requests
            self.rate_block.release()
        
        try:
            async with self.session.get(url, params=params, headers=self.foursquare_header) as request:

                if request.status == 403:
                    if not ignore_lock:
                        await self.rate_block.acquire()
                    else:
                        warn(f"Foursquare is being rate-limited; trying again in {ceil(fail_delay)} second(s")
                    await asyncio.sleep(ceil(fail_delay))
                    return await self.make_request(endpoint, *args, fail_delay=min(fail_delay * 2, 1800), ignore_lock=True, **params)
                if self.rate_block.locked():
                    self.rate_block.release()
                if request.status == 400:
                    stripped = dict(params)
                    stripped['client_id'] = '...'
                    stripped['client_secret'] = '...'
                    warn(f"Got response invalid error from Foursquare. Skipping.\n" + str(stripped))
                    return
                if request.status >= 500:
                    stripped = dict(params)
                    stripped['client_id'] = '...'
                    stripped['client_secret'] = '...'
                    warn(f"Got response {request.status} from Foursquare; trying again in {ceil(fail_delay)} second(s)\n{stripped}")
                    await asyncio.sleep(ceil(fail_delay))
                    return await self.make_request(endpoint, *args, fail_delay=min(fail_delay * 2, 28800), **params)

                self.request_counter.track()
                return await request.json()
        except aiohttp.ClientConnectionError:
            await self.client.close()
            self.client = aiohttp.ClientSession(headers=self.foursquare_header)
            return await self.make_request(endpoint, *args, fail_delay=fail_delay, **params)
            

            
def category_hier(categories, prefix=[]):
    """Returns a Pandas dataframe containing the hierarchy of categories up to five levels."""
    result = []
    
    for category in categories:
        category = json_normalize(category).iloc[0]
        current_category = pd.Series(
            data=prefix + [category.shortName] + [np.nan] * (4 - len(prefix)),
            name=str(category.id),
            index=[
                'cat_level_1',
                'cat_level_2',
                'cat_level_3',
                'cat_level_4',
                'cat_level_5'
            ]
        )
        result.append(current_category)
        if subcategories := category.categories:
            result += category_hier(subcategories, prefix + [category.shortName])
            
    return result

async with aiohttp.ClientSession(headers=FoursquarePipelineStep.foursquare_header) as session:
    params = {}
    params['v'] = FoursquarePipelineStep.v
    params['client_id'] = secrets['4SQ_CLIENT_ID']
    params['client_secret'] = secrets['4SQ_CLIENT_SECRET']
    cat_url = 'https://api.foursquare.com/v2/venues/categories'
    async with session.get(cat_url, params=params) as request:
        response = (await request.json())['response']['categories']
        
FoursquarePipelineStep.categories = pd.DataFrame(category_hier(response))
FoursquarePipelineStep.categories


,cat_level_1,cat_level_2,cat_level_3,cat_level_4,cat_level_5
4d4b7104d754a06370d81259,Arts & Entertainment,NaN,NaN,NaN,NaN
56aa371be4b08b9a8d5734db,Arts & Entertainment,Amphitheater,NaN,NaN,NaN
4fceea171983d5d06c3e9823,Arts & Entertainment,Aquarium,NaN,NaN,NaN
4bf58dd8d48988d1e1931735,Arts & Entertainment,Arcade,NaN,NaN,NaN
4bf58dd8d48988d1e2931735,Arts & Entertainment,Art Gallery,NaN,NaN,NaN
...,...,...,...,...,...
52f2ab2ebcbc57f1066b8b51,Travel,Tram,NaN,NaN,NaN
54541b70498ea6ccd0204bff,Travel,Transportation Services,NaN,NaN,NaN
4f04b25d2fb6e1c99f3db0c0,Travel,Lounge,NaN,NaN,NaN
57558b36e4b065ecebd306dd,Travel,Truck Stop,NaN,NaN,NaN


In [6]:
class DatabasePipelineStep(PipelineStep, ABC):
    """Pipeline step for handle data for the database for this project.
    
    async_batches are disabled here because I just have a simple MariaDB instance, on a single
    disk, so running multiple queries will not likely run any faster--and may even run slower.
    
    Attributes
    ----------
    drop_tables : bool
        Whether to drop and re-create the tables before running the pipeline, essentially
        starting over. Defaults to false.
    """
    async_batches = False
    max_batch_size = 500
    
    engine = None
    meta = None
    
    def __init__(self, drop_tables=False):
        super().__init__()
        self.drop_tables = drop_tables
        
    async def setup(self):
         if self.meta:
            async with self.engine.acquire() as conn:
                if self.drop_tables:
                    await self.drop_all(conn)
                await self.create_all(conn)
                
    async def drop_all(self, conn):
        for table in self.meta.tables.values():
            try:
                await conn.execute(DropTable(table))
            except:
                # Table doesn't exist
                pass
                
    async def create_all(self, conn):
        for table in self.meta.tables.values():
            try:
                await conn.execute(table.select())
            except:
                await conn.execute(CreateTable(table))
            
DatabasePipelineStep.engine = await create_engine(**secrets['db_prod'], autocommit=True, echo=True)


# Defining the actual pipeline steps

Now, I can use the code I've written to create a data pipeline. To test, I will start with Fort Wayne, IN like I did in the previous document.

## Wikipedia "Extract" and "Transform" Steps

These steps are a bit intermingled because I'm using some of the data I obtain from the transformation of one city to start the extraction of data from more citites (i.e. in the same county)

In [7]:
class NormalizeCityNames(WikipediaPipelineStep):
    """First step: take incoming city names and normalize them according to the titles of
    their pages on Wikipedia.

    Changes batch size to 50 since Wikipedia supports _querying_ 50 pages at a time
    """

    batch_size = 49

    async def process_batch(self, city_names):
        params = {
            "action": "query",
            "format": "json",
            "redirects": 1,
            "titles": "|".join(city_names)
        }
        try:
            result = await self.make_request(params)
            response = result['query']
        except KeyError as e:
            raise RuntimeError(f"Response not valid: {result}", city_names)
        if 'redirects' in response:
            for redirect in response['redirects']:
                #TODO: Cache redirects
                yield redirect['to']
        for page in response['pages'].values():
            if "missing" in page.keys():
                warn(f"The city {page['title']} was provided but is not available on Wikipedia, and has been skipped")
            elif page['title'] in city_names:
                # Original name was not redirected; yield the original name
                yield page['title']

```python
>>> pipeline = DataPipeline(NormalizeCityNames(), data=['Fort Wayne'])
>>> await pipeline.run()
```
```
['Fort Wayne, Indiana']
```

In [8]:
class ParseTree(WikipediaPipelineStep):
    """
    For each incoming city name, attatch its Wikipedia parsetree
    """
    async def process_batch(self, normalized_city_names):
        for city in normalized_city_names:
            params = {
                "action": "parse",
                "format": "json",
                "redirects": 1,
                "prop": "parsetree",
                "page": city
            }
            try:
                result = await self.make_request(params)
                raw_response = result['parse']['parsetree']['*']
            except Exception as e:
                warn(f"Could not get parsetree from {city}:\n{result}")
                continue
            response = xml.canonicalize(raw_response, strip_text=True)
            tree = xml.fromstring(response)
            await self.remove_comments(tree)
            yield (city, tree)
            
    async def remove_comments(self, root):
        """Remove any `<comment>` children tags, as they can interfere with parsing"""
        for child in list(root):  # Create list of elements that doesn't change during iteration
            if child.tag == 'comment':
                root.remove(child)
            else:
                await self.remove_comments(child)

```python 
>>> pipeline = DataPipeline(NormalizeCityNames(), ParseTree(), data=['Fort Wayne'])
>>> await pipeline.run()
```
```
[('Fort Wayne, Indiana', <Element 'root' at 0x7ff547383f90>)]
```

In [9]:
class GetCitiesFromCounties(WikipediaPipelineStep):
    """
    Figure out which of a city's navboxes is for the county, and use that to expand each
    city into the cities in its county.

    Leaves city and tree attached so that the tree for the original city is not obtained twice.
    """
    template_cache = {}
    
    async def process_batch(self, city_parsetrees):
        for original_city, tree in city_parsetrees:
            seat = None
            cities = None
            state = None
            templates = self.get_navbox_templates(tree)
            state_template, county_template = await self.filter_templates(templates)
            
            try:
                raw_response, template_page = await self.get_template_page(state_template)
                state = self.get_state(template_page)
            except:
                pass
            
            try:
                raw_response, template_page = await self.get_template_page(county_template)
                cities = self.parse_cities(raw_response)
                seat = await self.get_seat(template_page)
            except:
                pass
            
            if cities:
                for city in cities:
                    yield (original_city, tree, city, state, seat)
            else:
                yield original_city, tree, original_city, state, seat

    def get_navbox_templates(self, wiki_page_tree):
        """Finds the topic navigation boxes on the wiki page (usually at the bottom)"""
        navboxes = wiki_page_tree.findall(".//template[title='Navboxes']/part[name='list']/value/template/title")
        if len(navboxes) == 0:
            navboxes = wiki_page_tree.findall(".//template/title")
        return ['Template:{}'.format(elem.text) for elem in navboxes]
    
    async def filter_templates(self, templates):
        """Coroutine that uses Wikipedia API to filter templates into state and county.
        
        Returns
        -------
        tuple length 2 of strings
        """
        if len(templates) == 0:
            return None, None
        params = {
            "action": "query",
            "format": "json",
            "redirects": 1,
            "prop": "templates",
            "titles": "|".join(templates[-50:]),
            "tltemplates": "Template:US county navigation box|Template:US state navigation box"
        }
        try:
            result = await self.make_request(params)
            pages = result['query']['pages']
        except Exception as e:
            raise Exception(f"Invalide request with titles {params['titles']}.\n{result}") from e
        state_template = None 
        county_template = None
        for page in pages.values():
            if 'templates' not in page:
                continue
            for template in page['templates']:
                if template['title'] == 'Template:US state navigation box':
                    state_template = page['title']
                elif template['title'] == 'Template:US county navigation box':
                    county_template = page['title']
        return state_template, county_template

    async def get_template_page(self, template):
        """Gets the template that defines the navigation box.
        
        Returns
        -------
        tuple[str, ElementTree]
            The first element is the raw XML returned by the WikiPedia API.
            The second element is the stripped and parsed XML data.
        """
        if template in self.template_cache:
            return self.template_cache[template]
        params = {
            "action": "parse",
            "format": "json",
            "redirects": 1,
            "prop": "parsetree",
            "page": template
        }
        raw_response = (await self.make_request(params))['parse']['parsetree']['*']
        response = xml.canonicalize(raw_response, strip_text=True)
        self.template_cache[template] = raw_response, xml.fromstring(response)
        return self.template_cache[template]
        
    async def get_seat(self, root):
        """Gets the seat (e.g. metropolis) for a specified area from the navbox"""
        seat_name = root.find(".//template[title='US county navigation box']/part[name='seat']/value").text
        params = {
            "action": "query",
            "format": "json",
            "redirects": "1",
            "titles": seat_name
        }
        response = await self.make_request(params)
        page = list(response['query']['pages'].values())[0]
        assert 'missing' not in page
        return page['title']

        
    def parse_cities(self, raw_response_txt):
        """Using the raw XML, gets the cities from the navbox because they will be formatted as a list."""
        listed_city = re.compile(r"""
            ^\* \ *       # Line starts with "*" plus any number of spaces
            \[{2}         # Start of link "[["
                ([^\|]+)  # First part of link (between "[[" and "|"). This is the part that gets captured.
                \|        # Separator "|"
                [^\|]+    # Second part of link (between "|" and "]]")
            \]{2}‡?       # End of link "]]" plus optional ‡ character
            \ *$          # End with any number of spaces
        """, re.VERBOSE + re.MULTILINE)
        listed_cities = listed_city.findall(raw_response_txt)
        return listed_cities
    
    def get_state(self, root):
        """Gets the canonical state name for a city from the navbox"""
        return root.find(".//template[title='US state navigation box']/part[name='template_name']/value").text

```python
>>> pipeline = DataPipeline(
...     NormalizeCityNames(), 
...     ParseTree(), 
...     GetCitiesFromCounties(), 
...     data=['Fort Wayne']
... )
...
>>> results = await pipeline.run()
>>> results[:3]
```

```
[('Fort Wayne, Indiana',
  <Element 'root' at 0x7ff546a30d60>,
  'Fort Wayne, Indiana',
  'Indiana',
  'Fort Wayne, Indiana'),
 ('Fort Wayne, Indiana',
  <Element 'root' at 0x7ff546a30d60>,
  'New Haven, Indiana',
  'Indiana',
  'Fort Wayne, Indiana'),
 ('Fort Wayne, Indiana',
  <Element 'root' at 0x7ff546a30d60>,
  'Woodburn, Indiana',
  'Indiana',
  'Fort Wayne, Indiana')]
```

In [10]:
normalize = NormalizeCityNames()
parse = ParseTree()

class SecondNormalizeStep(NormalizeCityNames):
    """Subclass of NormalizeCityNames, which is modified to accept and yield back the city's county seat"""
    max_batch_size = 49
    def __init__(self, first_step):
        super().__init__()
        self._duplicate_cache = first_step._duplicate_cache
    
    async def process_batch(self, batch):
        cities, states, seats = zip(*batch)
        i = 0
        async for normalized_city in super().process_batch(cities):
            yield normalized_city, states[i], seats[i]
            i += 1
            
    def is_duplicate(self, record):
        city = record[0]
        return super().is_duplicate(city)
        
    
class SecondParseTreeStep(ParseTree):
    """Subclass of ParseTree, which is modified to accept and yield back the city's county seat"""
    def __init__(self, first_step):
        super().__init__()
        self._duplicate_cache = first_step._duplicate_cache
        
    async def process_batch(self, batch):
        norm_cities, states, seats = zip(*batch)
        i = 0
        async for city_name, tree in super().process_batch(norm_cities):
            yield city_name, states[i], seats[i], tree
            
    def is_duplicate(self, record):
        city = record[0]
        return super().is_duplicate(city)
        

class NewCitySplit(PipelineStep):
    """Runs the same processing on new cities in the county.
    
    It's designed specifically not to process the original city by creating a "split":
    
        incoming city matches original?
          |          \\
         yes          no
          |            \\
          |             |
          |             v
          |         normalize
          |             |
          |             v
          |           parse
          |             |
          v             v
         yield        yield
           \\            /
            \\          /
             \\        /
             |       |
             v       v
         NewCitySplit.outputs
    
    """
    def __init__(self, normalize_step, parse_step):
        super(NewCitySplit, self).__init__()
        self.normalize_step = SecondNormalizeStep(normalize_step)
        self.parse_step = SecondParseTreeStep(parse_step)
        self.normalize_step.attach(self.parse_step)
        self.parse_step.outputs = self.outputs
    
    async def process_batch(self, batch):
        for original_city, tree, city, state, seat in batch:
            if city == original_city:
                yield city, state, seat, tree
            else:
                self.normalize_step.put((city, state, seat))
        # Don't mark this task as "complete" until the pipeline step are done
        await self.normalize_step.join()
        await self.parse_step.join()
    
    def start(self):
        super().start()
        self.normalize_step.start()
        self.parse_step.start()
        
        
    def stop(self):
        super().stop()
        self.normalize_step.stop()
        self.parse_step.stop()
                
        

```python
>>> normalize = NormalizeCityNames()
>>> parse = ParseTree()
>>>
>>> pipeline = DataPipeline(
...     normalize, 
...     parse, 
...     GetCitiesFromCounties(), 
...     NewCitySplit(normalize, parse), 
...     data=['Fort Wayne, IN']
... )
... 
>>> wikipedia_results = await pipeline.run()
>>> wikipedia_results[:5]
```

```
[('Fort Wayne, Indiana',
  'Indiana',
  'Fort Wayne, Indiana',
  <Element 'root' at 0x7ff546932f90>),
 ('Aboite, Indiana',
  'Indiana',
  'Fort Wayne, Indiana',
  <Element 'root' at 0x7ff546890400>),
 ('Aboite Township, Allen County, Indiana',
  'Indiana',
  'Fort Wayne, Indiana',
  <Element 'root' at 0x7ff546897b30>),
 ('Academie, Indiana',
  'Indiana',
  'Fort Wayne, Indiana',
  <Element 'root' at 0x7ff5468a2360>),
 ('Adams Township, Allen County, Indiana',
  'Indiana',
  'Fort Wayne, Indiana',
  <Element 'root' at 0x7ff54689b950>)]
```

In [11]:
class WikiParsing(PipelineStep):
    """
    Extracts data from XML ElementTree to put into database or use with FourSquare API.
    
    Methods
    -------
    put(record):
        Adds a record. The record should be a Tuple of the following elements:
        
            1. city (string): The canonical name of the city; e.g. `'New Haven, Indiana'`
            2. state (string): The name of the state; e.g. `'Indiana'`
            3. seat (string): The county seat; e.g. `'Fort Wayne, Indiana'`
            4. tree (xml.etree.ElementTree): The parsed XML from Wikipedia for the city
    """
    async_batches = True
    batch_size = None
    
    async def process_batch(self, cities):
        for city, state, seat, tree in cities:
            try:
                latitude, longitude = self.parse_settlement_coords(tree, city)
                yield (city, {
                    'city_name': city,
                    'metro_name': seat,
                    'state_name': state,
                    'center_latitude': latitude,
                    'center_longitude': longitude,
                    'area_val': self.infobox_value(tree, "area_total_sq_mi", float),
                    'city_population': self.infobox_value(tree, "population_est", int),
                    'population_density': self.infobox_value(tree, "population_density_sq_mi", float)
                }, self.get_weather_table(tree) if city == seat else None, self.get_population_history(tree, city))
            except Exception as e:
                raise Exception("Error on record: " + str((city, state, seat, tree))) from e
            
    def is_duplicate(self, record):
        city = record[0]
        return super().is_duplicate(city)
    
    def parse_settlement_coords(self, wiki_data, city=''):
        try:
            coords = wiki_data.findall(".//part[name='coordinates']/value/template[title='coord']/part/value")
            if coords == None or len(coords) == 0:
                return (None, None) 
            # Check Lat/Lng already in decimal form
            if '.' in coords[0].text:
                if coords[1].text not in 'NSEW':
                    return tuple([float(x.text) for x in coords[:2]])
                lat_abs, lat_pole, lng_abs, lng_pole = [x.text for x in coords[:4]]
                latitude = float(lat_abs) * 1 if lat_pole == 'N' else -1
                longitude = float(lng_abs) * 1 if lng_pole == 'E' else -1
                return latitude, longitude
            # Convert from DMS (degrees, minutes, seconds) to Decimal
            if coords[2].text in 'NSEW':
                lat_deg, lat_min, lat_pole, lng_deg, lng_min, lng_pole = [x.text for x in coords[:6]]
                lat_sec = 0
                lng_sec = 0
            else:
                lat_deg, lat_min, lat_sec, lat_pole, lng_deg, lng_min, lng_sec, lng_pole = [x.text for x in coords[:8]]
            lat_sign = 1 if lat_pole == 'N' else -1
            lng_sign = 1 if lng_pole == 'E' else -1
            latitude = sum([float(lat_deg), float(lat_min)/60.0, float(lat_sec)/3600.0]) * lat_sign
            longitude = sum([float(lng_deg), float(lng_min)/60.0, float(lng_sec)/3600.0]) * lng_sign
            return latitude, longitude
        except Exception as e:
            warnings.warn(f"Could not parse coordinates for {city}", source=e)
            return None, None
    
    def infobox_value(self, tree, part_name, astype=str):
        template = tree.find(".//template[title='Infobox settlement']")
        if template == None:
            return None
        value = template.find(".part[name='{}'].value".format(part_name))
        if value == None or value.text == None:
            return None
        
        if astype in [int, float]:
            txt = value.text.replace(',', '')
            try:
                return astype(txt)
            except Exception as e:
                warn(f"Could not convert '{txt}' into number for part {part_name}")
                return
        return astype(value.text)
    
    def get_weather_table(self, tree):
        weather_box = tree.find(".//template[title='Weather box']")
        if weather_box is None:
            return None
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'year']
        stat_names = {
            'record high F': 'record_high',
            'avg record high F': 'avg_record_high',
            'high F': 'avg_high',
            'low F': 'avg_low',
            'avg record low F': 'avg_record_low',
            'record low F': 'record_low',
            'precipitation inch': 'avg_precip',
            'snow inch': 'avg_snow',
            'precipitation days': 'precip_days',
            'snow days': 'snow_days',
            'sun': 'sunshine_hours',
            'percentsun': 'daily_sunshine'
        }

        series_list = []
        for month in months:
            data = {}
            for stat in stat_names.keys():
                elem = weather_box.find(f".//part[name='{month} {stat}'].value")
                if elem is None or elem.text is None:
                    val = None
                else:
                    val = float(elem.text.replace('−', '-')) # The dashes in the data are not standard dashes for some reason
                data[stat_names[stat]] = val
            ix = pd.Index(data.keys(), name='weather_month')
            series_list.append(
                pd.Series(data=data.values(), index=ix, name=month)
            )
        return pd.DataFrame(series_list)
    
    def get_population_history(self, tree, city_name):
        assert city_name is not None
        census = tree.find(".//template[title='US Census population']")
        if census is None:
            return

        data = []

        for part in census.findall("part"):
            year = part.find("name").text
            if year and year.isnumeric() and len(year) == 4 and part.find("value").text:
                data.append((city_name, year, int(part.find("value").text.replace(",", ""))))
        
        if len(data) == 0:
            return None
                
        return pd.DataFrame(data, columns=['city_name', 'census_year', 'census_population'])

Before starting the "load" step, I'm going to test the full pipeline.

The `%store` magic allows me to store the results and use them later on, instead of re-using the APIs again. This allows me to restart the notebook without losing data and needing to re-query 9.

I only run the pipeline if the data needed to continue does not exist after reading stored data via `%store -r`. If I want to test the pipeline again in the future, I can create a console for the notebook and call `%store -z`

In [12]:
%store -r
try:
    parse_results
except NameError:
    parsing_pipeline = DataPipeline(
        normalize, 
        parse, 
        GetCitiesFromCounties(),
        NewCitySplit(normalize, parse), 
        WikiParsing(),
        data=['Fort Wayne, IN']
    )
    parse_results = await parsing_pipeline.run()
    fort_wayne_data = parse_results[0][1]
    fort_wayne_weather_data = parse_results[0][2]
    fort_wayne_pop_history = parse_results[0][3]
    %store parse_results fort_wayne_data fort_wayne_weather_data fort_wayne_pop_history

In [13]:
fort_wayne_data

{'city_name': 'Fort Wayne, Indiana',
 'metro_name': 'Fort Wayne, Indiana',
 'state_name': 'Indiana',
 'center_latitude': 41.080555555555556,
 'center_longitude': -85.13916666666667,
 'area_val': 110.79,
 'city_population': 270402,
 'population_density': 2445.46}

In [14]:
fort_wayne_weather_data

weather_month,record_high,avg_record_high,avg_high,avg_low,avg_record_low,record_low,avg_precip,avg_snow,precip_days,snow_days,sunshine_hours,daily_sunshine
Jan,69.0,53.5,32.4,17.4,-4.5,-24.0,2.26,10.1,12.6,9.5,148.5,50.0
Feb,73.0,56.9,36.3,20.3,0.5,-19.0,2.04,7.7,10.1,6.9,158.5,53.0
Mar,87.0,72.5,48.0,28.7,10.9,-10.0,2.71,4.1,12.2,4.1,206.3,56.0
Apr,90.0,81.0,61.1,38.9,23.4,7.0,3.52,1.0,12.9,1.0,251.4,63.0
May,97.0,86.6,71.7,49.2,35.4,27.0,4.27,0.0,13.0,0.0,311.9,69.0
Jun,106.0,93.0,80.9,59.3,46.2,36.0,4.16,0.0,10.9,0.0,340.0,75.0
Jul,106.0,93.4,84.4,62.7,51.6,38.0,4.24,0.0,9.8,0.0,347.0,76.0
Aug,102.0,91.7,82.2,60.8,49.3,38.0,3.64,0.0,9.4,0.0,318.2,75.0
Sep,100.0,88.9,76.0,52.6,38.2,29.0,2.80,0.0,9.1,0.0,258.1,69.0
Oct,91.0,81.0,63.4,41.8,27.8,19.0,2.84,0.3,9.7,0.2,207.6,60.0


In [15]:
fort_wayne_pop_history

,city_name,census_year,census_population
0,"Fort Wayne, Indiana",1850,4282
1,"Fort Wayne, Indiana",1860,10388
2,"Fort Wayne, Indiana",1870,17718
3,"Fort Wayne, Indiana",1880,26880
4,"Fort Wayne, Indiana",1890,35393
5,"Fort Wayne, Indiana",1900,45115
6,"Fort Wayne, Indiana",1910,63933
7,"Fort Wayne, Indiana",1920,86549
8,"Fort Wayne, Indiana",1930,114946
9,"Fort Wayne, Indiana",1940,118410


## Wikipedia "Load" Steps

This is where I take the data I've obtained using the Wikipedia API and transformed for my benefits, and load it into a database.

Quick note: I'm using the SQLAlchemy to generate SQL statements--not because I want to avoid writing the SQL, but I personally find that using Python to define the tables instead of SQL allows the code to limit itself to just one language. This helps foster code readability, especially with syntax highlighting.

In [16]:
class WikipediaDatabaseStep(DatabasePipelineStep):
    """Upload incoming parsed data to the database."""
    drop_duplicates = False
    
    meta = MetaData()
    
    async def process_batch(self, batch):
        cities, city_records, weather_tables, population_tables = zip(*batch)
        # No preprocessing needs to be done, so we can just yield the city names back
        for city in cities:
            yield city 
            
        await self.insert_cities(city_records)
        await self.insert_weather_data(cities, weather_tables)
        await self.insert_population_history(population_tables)
        
    city_table = Table("citites", meta,
        Column('city_name', String(50), primary_key=True, comment='City Name'),
        Column('metro_name', String(50), comment='Metropolitan Area Name'),
        Column('state_name', String(25), comment='State Name'),
        Column('center_latitude', Numeric(10, 6), comment='Latitude of City center'),
        Column('center_longitude', Numeric(10, 6), comment='Longitude of City center'),
        Column('area_val', Numeric(10, 4), comment='Area of city in square miles'),
        Column('city_population', Integer, comment='Total population of city'),
        Column('population_density', Numeric(10, 4), comment='Population Density per square mile')
    )
        
    async def insert_cities(self, city_records):
        async with self.engine.acquire() as conn:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", message=r".*Data truncated for column '.*")
                warnings.filterwarnings("ignore", r".*Duplicate entry .* for key.*")
                for record in city_records:
                    record.update({key: None for key in record.keys() if pd.isna(record[key])})
                await conn.execute(
                    insert(self.city_table, prefixes=['ignore']),
                    city_records
                )
                
    weather_table = Table(
        "metro_weather_data", meta,
        Column('city_name', String(50), primary_key=True, comment='City Name'),
        Column('weather_month', String(10), primary_key=True, comment='Month of weather conditions'),
        Column('record_high', Numeric(5, 2), comment='Record high temperature'),
        Column('avg_record_high', Numeric(5, 2), comment='Average highest temperature of month'),
        Column('avg_high', Numeric(5, 2), comment='Average daily high temperature of month'),
        Column('avg_low', Numeric(5, 2), comment='Average daily low temerature of month'),
        Column('avg_record_low', Numeric(5, 2), comment='Average lowest temperature of month'),
        Column('record_low', Numeric(5, 2), comment='Record low temperature'),
        Column('avg_precip', Numeric(7, 3), comment='Average precipitation inches'),
        Column('avg_snow', Numeric(7, 3), comment='Average snowfall inches'),
        Column('precip_days', Numeric(5, 2), comment='Average number of days where precipitation ≥0.1 inches'),
        Column('snow_days', Numeric(5, 1), comment='Average number of days where snowfall ≥0.1 inches'),
        Column('sunshine_hours', Numeric(7, 2), comment='Average number of hours with sunshine'),
        Column('daily_sunshine', Numeric(5, 2), comment='Average percent of daily sunshine')
    )
    
    async def insert_weather_data(self, cities, weather_tables):
        try:
            data_frame = pd.concat(weather_tables, keys=cities, names=['city_name']).reset_index()
        except ValueError:
            return  # No values passed to upload
        data_frame.columns.values[1] = 'weather_month'
        records = data_frame.to_dict('records')
        for record in records:
            record.update({key: None for key in record.keys() if pd.isna(record[key])})
        async with self.engine.acquire() as conn:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", r".*Duplicate entry .* for key.*")
                await conn.execute(
                    insert(self.weather_table, prefixes=['ignore']),
                    records
                )
            
    population_table = Table(
        "city_population_history", meta,
        Column('city_name', String(50), primary_key=True, comment='City Name'),
        Column('census_year', Integer(4, zerofill=True), primary_key=True, comment='Census Year'),
        Column('census_population', Integer, comment='Total recorded population')
    )
            
    async def insert_population_history(self, population_tables):
        try:
            data_frame = pd.concat(population_tables)
        except:
            return  # No values passed to upload
        if len(data_frame) == 0:
            return
        records = data_frame.to_dict('records')
        
        async with self.engine.acquire() as conn:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", r".*Duplicate entry .* for key.*")
                await conn.execute(
                    insert(self.population_table, prefixes=['ignore']),
                    records
                )

In [17]:
await DataPipeline(
    WikipediaDatabaseStep(drop_tables=True),
    data=parse_results
).run()

# Pandas does not support async engines (yet)
sync_engine = sa.create_engine(secrets['db_connection_string'])

print("Total Wikipedia requests made:", WikipediaPipelineStep.request_counter.total_count)


Total Wikipedia requests made: 0


In [18]:
pd.read_sql(WikipediaDatabaseStep.city_table.select(), sync_engine)

,city_name,metro_name,state_name,center_latitude,center_longitude,area_val,city_population,population_density
0,"Aboite Township, Allen County, Indiana","Fort Wayne, Indiana",Indiana,41.052222,-85.285278,33.35,NaN,1073.20
1,"Aboite, Indiana","Fort Wayne, Indiana",Indiana,41.000000,-85.318056,NaN,NaN,NaN
2,"Academie, Indiana","Fort Wayne, Indiana",Indiana,41.170833,-85.145833,NaN,NaN,NaN
3,"Adams Township, Allen County, Indiana","Fort Wayne, Indiana",Indiana,41.045833,-85.050556,NaN,NaN,NaN
4,"Allen, Indiana","Fort Wayne, Indiana",Indiana,41.200000,-85.195833,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
65,"Wayne Township, Allen County, Indiana","Fort Wayne, Indiana",Indiana,41.049444,-85.164722,NaN,NaN,NaN
66,"Woodburn, Indiana","Fort Wayne, Indiana",Indiana,41.126111,-84.852778,0.96,1639.0,1712.64
67,"Yoder, Indiana","Fort Wayne, Indiana",Indiana,40.931111,-85.176667,NaN,NaN,NaN
68,"Zanesville, Indiana","Fort Wayne, Indiana",Indiana,40.915833,-85.281389,0.84,621.0,750.00


In [19]:
pd.read_sql(WikipediaDatabaseStep.weather_table.select(), sync_engine)

,city_name,weather_month,record_high,avg_record_high,avg_high,avg_low,avg_record_low,record_low,avg_precip,avg_snow,precip_days,snow_days,sunshine_hours,daily_sunshine
0,"Fort Wayne, Indiana",Apr,90.0,81.0,61.1,38.9,23.4,7.0,3.52,1.0,12.9,1.0,251.4,63.0
1,"Fort Wayne, Indiana",Aug,102.0,91.7,82.2,60.8,49.3,38.0,3.64,0.0,9.4,0.0,318.2,75.0
2,"Fort Wayne, Indiana",Dec,71.0,56.0,36.2,22.1,1.9,-18.0,2.77,8.5,13.0,8.2,108.2,38.0
3,"Fort Wayne, Indiana",Feb,73.0,56.9,36.3,20.3,0.5,-19.0,2.04,7.7,10.1,6.9,158.5,53.0
4,"Fort Wayne, Indiana",Jan,69.0,53.5,32.4,17.4,-4.5,-24.0,2.26,10.1,12.6,9.5,148.5,50.0
5,"Fort Wayne, Indiana",Jul,106.0,93.4,84.4,62.7,51.6,38.0,4.24,0.0,9.8,0.0,347.0,76.0
6,"Fort Wayne, Indiana",Jun,106.0,93.0,80.9,59.3,46.2,36.0,4.16,0.0,10.9,0.0,340.0,75.0
7,"Fort Wayne, Indiana",Mar,87.0,72.5,48.0,28.7,10.9,-10.0,2.71,4.1,12.2,4.1,206.3,56.0
8,"Fort Wayne, Indiana",May,97.0,86.6,71.7,49.2,35.4,27.0,4.27,0.0,13.0,0.0,311.9,69.0
9,"Fort Wayne, Indiana",Nov,79.0,68.9,49.9,32.9,18.9,-1.0,3.09,1.8,11.2,2.6,124.2,42.0


In [20]:
pd.read_sql(WikipediaDatabaseStep.population_table.select(), sync_engine)

,city_name,census_year,census_population
0,"Fort Wayne, Indiana",1850,4282
1,"Fort Wayne, Indiana",1860,10388
2,"Fort Wayne, Indiana",1870,17718
3,"Fort Wayne, Indiana",1880,26880
4,"Fort Wayne, Indiana",1890,35393
...,...,...,...
73,"Woodburn, Indiana",2000,1579
74,"Woodburn, Indiana",2010,1520
75,"Zanesville, Indiana",1880,93
76,"Zanesville, Indiana",2000,602


---

## Foursquare "Extract" Steps

In [21]:
class ExpandFoursquareRequests(PipelineStep):
    """Generates the parameters of all of the Foursquare calls to be made"""
    async def process_batch(self, cities):
        city_tuples = list(product(['near'], cities))
        radii = list(product(['radius'], [1000, 25000, 100000]))
        queries = list(product(
            ['section'], 
            [
                'food', 
                'coffee',
                'shops', 
                'arts', 
                'outdoors', 
                'sights', 
                'topPicks'
            ]
        )) + list(product(
            ['query'], 
            [
                'hiking trail',
                'bbq',
                'historic sites',
                'park',
                'dog park',
                'british food',
                'arcade',
                'ice cream shop'
            ]
        ))
        for params in product(city_tuples, radii, queries):
            yield dict(params)

```python
>>> pipeline = DataPipeline(ExpandFoursquareRequests(), data=['Fort Wayne, IN'])
>>> await pipeline.run()
```
```
[{'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'food'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'drinks'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'coffee'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'shops'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'arts'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'outdoors'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'sights'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'trending'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'section': 'topPicks'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'query': 'hiking trail'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'query': 'bbq'},
 {'near': 'Fort Wayne, IN', 'radius': 1000, 'query': 'historic sites'},
 ...
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'bbq'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'historic sites'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'park'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'dog park'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'british food'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'irish food'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'arcade'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'pizzeria'},
 {'near': 'Fort Wayne, IN', 'radius': 100000, 'query': 'ice cream shop'}]
```


In [22]:
class GetFoursquareData(FoursquarePipelineStep):
    max_batch_size = 1
    async_batches = True
    drop_duplicates = False
    
    async def process_batch(self, batch):
        for original_params in batch:
            params = {
                **original_params,
                'limit': 50,
                'sortByPopularity': 1
            }
            response = await self.make_request('explore', **params)
            if response is not None:
                yield (response, original_params)

## Foursquare "Transform" Step

In [23]:
class ParseFoursquareData(FoursquarePipelineStep):
    max_batch_size = 1
    async_batches = True
    drop_duplicates = False
    
    async def process_batch(self, batch):
        for response, params in batch:
            venues = self.parse(response, params)
            if venues is not None:
                yield venues
    
    def parse(self, data, params):
        if 'groups' not in data['response']:
            return None
        params = params
        venues = json_normalize(data, ['response', 'groups', 'items'], sep='_')
        if len(venues) == 0:
            return None
        geo = json_normalize(data['response']['geocode'], sep='_').loc[0] # json_normalize returns single-index df
        geo.index = pd.Index(f'geo_{name}' for name in geo.index)
        
        venues.loc[:, geo.index] = geo.values
        venues['search_popularity'] = venues.index.values
        query_type = 'section' if 'section' in params else 'query'
        venues['query_type'] = query_type
        venues['query'] = params[query_type]
        venues['search_city'] = params['near']
        venues['search_radius'] = params['radius']
        venues = venues.reindex(venues.columns.union(['venue_id', 'venue_name', 'venue_location_lat', 'venue_location_lng', 'venue_location_crossStreet',
            'venue_location_city', 'venue_location_state', 'venue_delivery_id'], sort=False), axis=1, fill_value=None)
        
        
        def get_categories(row):
            if not row.venue_categories:
                return pd.Series(
                    [np.nan] * 5,
                    self.categories.columns,
                    name=row.name
                )
            return self.categories.loc[row.venue_categories[0]['id']]

        venues = venues.merge(
            venues.apply(get_categories, axis=1), 
            left_index=True,
            right_index=True
        )
        
        return venues

In [24]:
%store -r
try:
    foursquare_results
except:
    pipeline = DataPipeline(
        ExpandFoursquareRequests(),
        GetFoursquareData(),
        ParseFoursquareData(),
        data=['Fort Wayne, Indiana']
    )

    await pipeline.run()
    foursquare_results = pd.concat(pipeline.results, ignore_index=True)
    %store foursquare_results

foursquare_results

,referralId,reasons_count,reasons_items,venue_id,venue_name,venue_location_address,venue_location_lat,venue_location_lng,venue_location_labeledLatLngs,venue_location_postalCode,...,cat_level_3,cat_level_4,cat_level_5,flags_outsideRadius,venue_location_neighborhood,venue_events_count,venue_events_summary,venue_events_items,venue_location_isFuzzed,venue_location_isServiceAreaBusiness
0,e-5-4ba682a3f964a520745939e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4ba682a3f964a520745939e3,Starbucks,4201 Coldwater Blvd.,41.116194,-85.138506,"[{'label': 'display', 'lat': 41.11619417, 'lng...",46805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-5-4b59cd2df964a520db9828e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b59cd2df964a520db9828e3,Starbucks,301 E Coliseum Blvd,41.118540,-85.138275,"[{'label': 'display', 'lat': 41.11854, 'lng': ...",46805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e-5-4b5098bdf964a520c52827e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b5098bdf964a520c52827e3,Panera Bread,526 Coliseum Boulevard,41.117775,-85.132799,"[{'label': 'display', 'lat': 41.117775, 'lng':...",46805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e-5-4b5fa409f964a520e9c529e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4b5fa409f964a520e9c529e3,Dunkin',5767 Saint Joe Rd,41.132373,-85.100943,"[{'label': 'display', 'lat': 41.13237328790414...",46835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e-5-4bc455334cdfc9b6ae709821-4,0,"[{'summary': 'This spot is popular', 'type': '...",4bc455334cdfc9b6ae709821,Dunkin',8051 Coldwater Rd,41.153179,-85.137254,"[{'label': 'display', 'lat': 41.15317898155577...",46825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676,e-0-4c55c8e04623be9a292e0bf5-45,0,"[{'summary': 'This spot is popular', 'type': '...",4c55c8e04623be9a292e0bf5,Zesto,2225 Broadway,41.064372,-85.152197,"[{'label': 'display', 'lat': 41.06437222810140...",46802,...,Ice Cream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2677,e-0-4c1c0517fcf8c9b60167a90b-46,0,"[{'summary': 'This spot is popular', 'type': '...",4c1c0517fcf8c9b60167a90b,Edon Dairy Treat,104 S Michigan St,41.555674,-84.769435,"[{'label': 'display', 'lat': 41.55567363368158...",43518,...,Ice Cream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2678,e-0-4b8d74cdf964a520cefc32e3-47,0,"[{'summary': 'This spot is popular', 'type': '...",4b8d74cdf964a520cefc32e3,La Michoacana,1421 N Wells St,41.087312,-85.145893,"[{'label': 'display', 'lat': 41.087312, 'lng':...",46808,...,Ice Cream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2679,e-0-4c5f4e5990b2c9b692773a22-48,0,"[{'summary': 'This spot is popular', 'type': '...",4c5f4e5990b2c9b692773a22,Zesto,210 E Washington Center Rd,41.131710,-85.139029,"[{'label': 'display', 'lat': 41.13171016180453...",46825,...,Ice Cream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Foursquare "Load" Step

In [25]:
class FoursquareDatabaseStep(DatabasePipelineStep):
    """Uploads the results from Foursquare queries (expects collection of pandas.DataFrame instances)
    
    This also does just a bit more transformation to the incoming dataframes to split the data into
    the unique venues and a table linking venue IDs to their preceding search parameters.
    """
    meta = MetaData()
    drop_duplicates = False
    
    async def process_batch(self, batch):
        data_frame = pd.concat(batch, ignore_index=True)
        
        search_results_df = data_frame[['venue_id', 'search_city', 'search_radius', 'query_type', 'query', 'search_popularity']]
        venue_data_df = data_frame[[
            'venue_id', 'venue_name', 'venue_location_lat', 'venue_location_lng', 'venue_location_crossStreet',
            'venue_location_city', 'venue_location_state',
            'venue_delivery_id', 'cat_level_1', 'cat_level_2', 'cat_level_3', 'cat_level_4'
        ]].drop_duplicates('venue_id')
        
        # This is the end of the pipeline...nothing to yield, but still need "yield" to create iterator
        yield
        
        await self.put_results(search_results_df)
        await self.put_venues(venue_data_df)
        
    results_table = Table("city_foursquare_results", meta,
        Column('venue_id', String(24), nullable=False, comment='Foursquare Venue ID'),
        Column('search_city', String(128), nullable=False, comment='Search City'),
        Column('search_radius', Integer, nullable=False, comment='Radius in meters'),
        Column('query_type', String(20), nullable=False, comment='Type of query made to get results'),
        Column('query', String(100), nullable=False, comment='Query text'),
        Column('search_popularity', Integer, nullable=False, comment='Popularity in search results')
    )
    
    async def put_results(self, df):
        records = df.to_dict('records')
        async with self.engine.acquire() as conn:
            await conn.execute(
                self.results_table.insert(),
                records
            )
            
    venues_table = Table("foursquare_venues", meta, 
        Column('venue_id', String(24), primary_key=True, comment='Foursquare Venue ID'),
        Column('venue_name', String(255), nullable=False, comment='Venue name'),
        Column('venue_location_lat', Numeric(12, 8), nullable=False, comment='Venue Location Latitude'),
        Column('venue_location_lng', Numeric(12, 8), nullable=False, comment='Venue Location Longitude'),
        Column('venue_location_crossStreet', String(255), comment='Street Intersection of Venue Location'),
        Column('venue_delivery_id', String(40), comment='Venue Delivery Identifier'),
        Column('venue_location_city', String(100), comment='City Name of Venue Location'),
        Column('venue_location_state', String(25), comment='State Code of Venue Location'),
        Column('cat_level_1', String(50), comment='Level 1 Category Name'),
        Column('cat_level_2', String(50), comment='Level 2 Category Name'),
        Column('cat_level_3', String(50), comment='Level 3 Category Name'),
        Column('cat_level_4', String(50), comment='Level 4 Category Name')
    )
    
    async def put_venues(self, df):
        df.fillna(sa.null())
        records = df.to_dict('records')
        for record in records:
            record.update({key: None for key in record.keys() if pd.isna(record[key])})
        async with self.engine.acquire() as conn:
             with warnings.catch_warnings():
                warnings.filterwarnings("ignore", r".*Duplicate entry .* for key.*")
                warnings.filterwarnings("ignore", message=r".*Data truncated for column '.*")
                await conn.execute(
                    insert(self.venues_table, prefixes=['ignore']),
                    records
                )

# Running the final pipeline

I have all my pipelines set up; now let's test a full pipeline.

Now, there's one more piece to load, and that's running the pipeline for all of the potential cities. I'm going to be using the [Data Science Job dataset](https://www.kaggle.com/andrewmvd/data-scientist-jobs) from andrewmvd on Kaggle.

In [26]:
!pip install --quiet --upgrade kaggle
# Note that kaggle API credentials need to be set up
!kaggle datasets download -p data --unzip -d andrewmvd/data-scientist-jobs

 67%|█████████████████████████▍            | 3.00M/4.49M [00:00<00:00, 11.5MB/s]
100%|██████████████████████████████████████| 4.49M/4.49M [00:00<00:00, 12.4MB/s]


In [27]:
data_science_df = pd.read_csv('data/DataScientist.csv', index_col='index')
data_science_df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
index,,,,,,,,,,,,,,,,
0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,3904,AWS Data Engineer,$55K-$112K (Glassdoor est.),About Us\n\nTachyon Technologies is a Digital ...,4.4,Tachyon Technologies\n4.4,"Dublin, OH","Irving, TX",201 to 500 employees,2011,Company - Private,IT Services,Information Technology,$10 to $25 million (USD),-1,-1
4376,3905,Data Analyst â Junior,$55K-$112K (Glassdoor est.),"Job description\nInterpret data, analyze resul...",5.0,"Staffigo Technical Services, LLC\n5.0","Columbus, OH","Woodridge, IL",51 to 200 employees,2008,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4377,3906,Security Analytics Data Engineer,$55K-$112K (Glassdoor est.),Job DescriptionThe Security Analytics Data Eng...,3.8,"PDS Tech, Inc.\n3.8","Dublin, OH","Irving, TX",5001 to 10000 employees,1977,Company - Private,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),-1,-1


In [28]:
salary_parser = re.compile(r"\$(\d*)K.\$(\d*)K.*")

def get_estimate(salary_txt):
    keys = ["lower_estimate", "upper_estimate"]
    parsed = salary_parser.findall(salary_txt)
    if not parsed:
        return pd.Series([None, None], keys)
    else:
        return pd.Series(parsed[0], keys)

data_science_df = data_science_df.merge(data_science_df['Salary Estimate'].apply(get_estimate), left_index=True, right_index=True)


In [29]:
data_science_df[['Job Title', 'Salary Estimate', 'Location', 'lower_estimate', 'upper_estimate']][pd.isna(data_science_df.lower_estimate)]

,Job Title,Salary Estimate,Location,lower_estimate,upper_estimate
index,,,,,
793,"Clinical Laboratory Scientist, Per Diem",$34-$53 Per Hour(Glassdoor est.),"Los Angeles, CA",None,None
794,Sr. Data Analyst,$34-$53 Per Hour(Glassdoor est.),"Burbank, CA",None,None
795,"Lead Data Science Instructor, Data Scientist",$34-$53 Per Hour(Glassdoor est.),"Los Angeles, CA",None,None
796,"Clinical Laboratory Scientist (Per Diem, Varia...",$34-$53 Per Hour(Glassdoor est.),"Monterey Park, CA",None,None
797,Data Engineer,$34-$53 Per Hour(Glassdoor est.),"Woodland Hills, CA",None,None
798,Clinical Laboratory Scientist - Generalist,$34-$53 Per Hour(Glassdoor est.),"Monrovia, CA",None,None
799,"Clinical Laboratory Scientist , Med Tech",$34-$53 Per Hour(Glassdoor est.),"Los Angeles, CA",None,None
1440,Stafford Data Science Tutor Jobs,$10-$26 Per Hour(Glassdoor est.),"Stafford, TX",None,None
1441,Spring Data Science Tutor Jobs,$10-$26 Per Hour(Glassdoor est.),"Spring, TX",None,None


I'm pretty content not including hourly jobs in my models. So I'm going to go ahead and put this into a table as well and use the "Location" field to start running the pipeline.

In [30]:
data_science_df.to_sql('data_science_jobs', sync_engine, if_exists='replace')

cities = data_science_df.Location.unique()
cities

array(['New York, NY', 'Lyndhurst, NJ', 'Brooklyn, NY', 'Jersey City, NJ',
       'Carle Place, NY', 'Newark, NJ', 'Franklin Lakes, NJ',
       'Port Washington, NY', 'Rockville Centre, NY',
       'Queens Village, NY', 'Fort Lee, NJ', 'Middletown, NJ',
       'Florham Park, NJ', 'Summit, NJ', 'Maywood, NJ', 'West Orange, NJ',
       'Los Angeles, CA', 'Woodland Hills, CA', 'Universal City, CA',
       'Anaheim, CA', 'Culver City, CA', 'Cerritos, CA',
       'Santa Monica, CA', 'Torrance, CA', 'El Segundo, CA',
       'Glendale, CA', 'Brea, CA', 'Burbank, CA', 'Westmont, CA',
       'Pasadena, CA', 'Irwindale, CA', 'Calabasas, CA',
       'Hermosa Beach, CA', 'Whittier, CA', 'Northridge, CA',
       'Marina del Rey, CA', 'Monaco, CA', 'Venice, CA',
       'Sherman Oaks, CA', 'Duarte, CA', 'Wilmington, CA',
       'Long Beach, CA', 'Monrovia, CA', 'Carson, CA', 'Seal Beach, CA',
       'Monterey Park, CA', 'Chicago, IL', 'Evanston, IL',
       'Melrose Park, IL', 'Lemont, IL', 'Rosemont

In [31]:
# For some reason, my PC restarted in the middle of running this pipeline, so I wrote this class so I don't have to start over from scratch
class SkipCompletedRequests(DatabasePipelineStep):
    max_batch_size = 1
    async_batches = False
    drop_duplicates = False
    
    async def process_batch(self, batch):
        for params in batch:
            query_param = params['query'] if 'query' in params else params['section']
            
            tbl = FoursquareDatabaseStep.results_table
            city_name = tbl.c.search_city
            radius = tbl.c.search_radius
            query_col = tbl.c.query

            qry = select([count(city_name.distinct())])\
                .where(city_name == params['near'])\
                .where(radius == params['radius'])\
                .where(query_col == query_param)
            
            async with self.engine.acquire() as conn:
                cur = await conn.execute(qry)
                result = await cur.scalar()
            
            if result == 0:
                yield params
            

In [32]:
normalize = NormalizeCityNames()
parse = ParseTree()

final_steps = [
    normalize,
    parse,
    GetCitiesFromCounties(),
    NewCitySplit(normalize, parse),
    WikiParsing(),
    WikipediaDatabaseStep(),
    ExpandFoursquareRequests(),
    SkipCompletedRequests(),
    GetFoursquareData(),
    ParseFoursquareData(),
    FoursquareDatabaseStep()
]

pipeline = DataPipeline(final_steps, data=cities)

In [33]:
import sys

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    pipeline.start()
    started = datetime.now()
    reprint = datetime.now() + timedelta(seconds=5)

    try:
        while not pipeline.done:
            if datetime.now() >= reprint:
                progress = []
                for step in pipeline.steps:
                    q_items = step.data.qsize()
                    tasks_running = len([task for task in step._all_tasks if not task.done()])
                    progress.append(str(q_items + tasks_running))
                    
                print(f"[{str(datetime.now() - started)}]")
                print("Pipeline progress:", '\t'.join(progress))
                print("Wikipedia requests made:", WikipediaPipelineStep.request_counter.total_count)
                print("Foursquare requests made:", FoursquarePipelineStep.request_counter.total_count)
                print()
                reprint = datetime.now() + timedelta(seconds=120)
            await asyncio.sleep(0)
    finally:
        pipeline.stop()

[0:00:05.099631]
Pipeline progress: 171	5	5	86	2	6	4	491	1	1	0
Wikipedia requests made: 75
Foursquare requests made: 0

[0:02:05.104590]
Pipeline progress: 0	0	0	145	0	1	2	42725	0	0	0
Wikipedia requests made: 1590
Foursquare requests made: 7

[0:04:05.124741]
Pipeline progress: 0	0	0	145	2	0	1	90639	0	0	0
Wikipedia requests made: 2681
Foursquare requests made: 16

[0:06:05.143332]
Pipeline progress: 0	0	0	145	0	2	0	133585	1	1	0
Wikipedia requests made: 3655
Foursquare requests made: 22

[0:08:05.153951]
Pipeline progress: 0	0	0	0	1	0	1	133089	0	0	1
Wikipedia requests made: 3666
Foursquare requests made: 23

[0:10:05.170781]
Pipeline progress: 0	0	0	0	0	0	0	132101	0	0	0
Wikipedia requests made: 3666
Foursquare requests made: 35

[0:12:05.185813]
Pipeline progress: 0	0	0	0	0	0	0	131156	0	0	0
Wikipedia requests made: 3666
Foursquare requests made: 42

[0:14:05.201325]
Pipeline progress: 0	0	0	0	0	1	0	130253	1	1	0
Wikipedia requests made: 3666
Foursquare requests made: 44

[0:16:05.224378]

In [34]:
pd.read_sql(FoursquareDatabaseStep.results_table.select(), sync_engine)

Exception during reset or similar
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 2212, in run_callable
    return conn.run_callable(callable_, *args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1653, in run_callable
    return callable_(self, *args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/sqlalchemy/dialects/mysql/base.py", line 2599, in has_table
    self.identifier_preparer._quote_free_identifiers(
  File "/opt/conda/lib/python3.8/site-packages/sqlalchemy/dialects/mysql/base.py", line 2314, in _quote_free_identifiers
    return tuple([self.quote_identifier(i) for i in ids if i is not None])
  File "/opt/conda/lib/python3.8/site-packages/sqlalchemy/dialects/mysql/base.py", line 2314, in <listcomp>
    return tuple([self.quote_identifier(i) for i in ids if i is not None])
  File "/opt/conda/lib/python3.8/site-packages/sqlalchemy/sql/compiler.py", line 3604,

,venue_id,search_city,search_radius,query_type,query,search_popularity
0,4b66def6f964a5202f2e2be3,"Lyndhurst, New Jersey",1000,section,food,0
1,4b9a9f22f964a5201dc735e3,"Lyndhurst, New Jersey",1000,section,food,1
2,4b44d7e7f964a520e6fd25e3,"Lyndhurst, New Jersey",1000,section,food,2
3,4bbe1522c6a2ef3b8a43ddbd,"Lyndhurst, New Jersey",1000,section,food,3
4,4bdb4c9a63c5c9b623432768,"Lyndhurst, New Jersey",1000,section,food,4
...,...,...,...,...,...,...
4513629,4bae5435f964a52068a33be3,"York Center, Ohio",100000,query,ice cream shop,9
4513630,4c372dda04cbb7131a42ee0d,"York Center, Ohio",100000,query,ice cream shop,10
4513631,4b91a65af964a520d6cc33e3,"York Center, Ohio",100000,query,ice cream shop,11
4513632,4dd6e9f652b1a5c6443fd34f,"York Center, Ohio",100000,query,ice cream shop,12


In [35]:
pd.read_sql(FoursquareDatabaseStep.venues_table.select(), sync_engine)

,venue_id,venue_name,venue_location_lat,venue_location_lng,venue_location_crossStreet,venue_delivery_id,venue_location_city,venue_location_state,cat_level_1,cat_level_2,cat_level_3,cat_level_4
0,3fd66200f964a52000e71ee3,Fat Cat,40.733665,-74.002950,btwn 7th Ave S & Bleecker St,None,New York,NY,Arts & Entertainment,Music Venue,Jazz Club,None
1,3fd66200f964a52000f11ee3,Melody Lanes,40.652726,-74.002993,at 5th Ave.,None,Brooklyn,NY,Arts & Entertainment,Bowling Alley,None,None
2,3fd66200f964a52002ef1ee3,The Short Stop,34.075293,-118.253701,at Sutherland St,None,Los Angeles,CA,Nightlife,Bar,None,None
3,3fd66200f964a52007f11ee3,Poquito Mas,34.159801,-118.331202,at Naomi St.,None,Burbank,CA,Food,Mexican,None,None
4,3fd66200f964a52008e81ee3,Serendipity 3,40.761758,-73.965054,btwn 2nd & 3rd Ave,None,New York,NY,Food,Desserts,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
228487,600335afb50b926ccc98e77d,Cultura Coffee Shop,29.326754,-98.551674,None,None,San Antonio,TX,Food,Coffee Shop,None,None
228488,600343989a58fc0b07106204,Redline Athletics,40.233030,-75.225066,None,None,Montgomeryville,PA,Outdoors & Recreation,Athletics & Sports,None,None
228489,6004a774cf4c6c1531cabe42,Haddonfield Zoo Sculpture Park,39.900356,-75.027940,None,None,Haddonfield,NJ,Outdoors & Recreation,Park,None,None
228490,6004b020c6f7954a4788243f,Garnet Stream,39.919204,-75.361948,None,None,Springfield,PA,Outdoors & Recreation,Park,None,None
